In [4]:
import hashlib

class sha_hash():
    def __init__(self, key, hashlen):
        self.key = key
        
        der = sum([ord(i) for i in self.key])
        self.hashstart = der % 64
        self.hashlen = hashlen
        
        asciichar = chr(33 + (der % 93))
        asciimult = der % 13

        self.keyx1 = self.sha_it(self.key[::-1])[:4]
        self.keyx2 = asciichar * asciimult
        self.keyx3 = key

    @staticmethod
    def sha_it(s):
        b = hashlib.sha256()
        b.update(s.encode())
        return b.hexdigest().upper()
        
    def hashid(self, id):
        a = '13' + self.keyx1 + id + self.keyx2 + self.keyx3
        return (self.sha_it(a) * 2)[self.hashstart:][:self.hashlen]
        
    def test_key(self):
        x = []
        for i in range(1000):
            for j in range(1000):
                subjid = str(i).zfill(3) + '-' + str(j).zfill(3)

                x.append(self.hashid(subjid))

        return len(set(x)) == len(x)

In [5]:
h = sha_hash("Dupa", 12)

In [8]:
h.hashid('101-017')

'E1CAD3BD6FFD'

In [7]:
h.hashid('101-002')

'34FDB6E7AA7E'

In [ ]:
'''
SAS CODE
=================
%macro mhashid(id, key, keylen);
  %let keyx1 = %substr(%sysfunc(sha256(%cmpres(%sysfunc(reverse(&key)))), $hex64.),1,4);

  %let s=0;
  %do i =1 %to %length(&key);
    %let s = %eval(&s+%sysfunc(rank(%substr(&key,&i,1))));
  %end;
  %let start = %sysfunc(mod(&s,64));
  %let asc = %sysfunc(byte(%eval(33+%sysfunc(mod(&s,93)))));
  %let ascn = %sysfunc(mod(&s,13));
  %let keyx2 = %sysfunc(repeat(%str(&asc),&ascn-1));

  %let sha=%sysfunc(repeat(%sysfunc(sha256(%str(13&keyx1.&id.&keyx2.&key)), $hex64.),1));
  %let hash = %substr(&sha,%eval(&start+1),&keylen);
  &hash;

%mend mhashid;

%let h = %mhashid(101-001, Dupa, 12);
%put &h;


%macro hashid(id, key, keylen);
  _X_keyx1 = substr(put(sha256(compress(reverse(&key))), $hex64.),1,4);
  _X_s=0;
  do _X_i =1 to length(&key);
    _X_s = _X_s+rank(substr(&key,_X_i,1));
  end;
  _X_start=mod(_X_s,64);
  _X_asc = byte(33 + mod(_X_s,93));
  _X_ascn = mod(_X_s,13);
  _X_sha=repeat(put(sha256('13'||compress(_X_keyx1)||compress(&id)||repeat(_X_asc,_X_ascn-1)||compress(&key)), $hex64.),1);
  hash = substr(_X_sha,_X_start+1,&keylen);
  drop _X_:;
%mend;

data a;
  do i=1 to 20;
    subjid = '101-'||put(i,z3.);
    %hashid(subjid, "Dupa", 12);
    output;
  end;
run;


'''